In [1]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [21]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [2]:
# Combine CSV, generate output

files = glob.glob("Clean_data/Kickstarter.csv")

df = pd.read_csv("Clean_data/Kickstarter.csv", index_col=None, header=0)

# csv = []

# for file in files:
#     df = pd.read_csv(file, index_col=None, header=0)
#     csv.append(df)

# df = pd.concat(csv, axis=0, ignore_index=True)

In [3]:
df.head(2)

,Unnamed: 0,backers_count,blurb,category,converted_pledged_amount,country,created_at,current_currency,deadline,fx_rate,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,1,1,With your help we will create this device that...,"{""id"":331,""name"":""3D Printing"",""slug"":""technol...",1,ES,2015-08-18 21:01:19,USD,2016-07-09 20:11:23,1.212886,...,save-water-100-liquid-downloads-in-toilets,https://www.kickstarter.com/discover/categorie...,False,False,failed,2016-07-09 20:11:24,1.136930,"{""web"":{""project"":""https://www.kickstarter.com...",1.136930,domestic
1,2,2,We at Ormiston Primary are looking at starting...,"{""id"":309,""name"":""Farms"",""slug"":""food/farms"",""...",9,NZ,2015-08-11 18:04:21,USD,2015-09-11 15:55:41,0.723585,...,ormiston-primary-community-garden,https://www.kickstarter.com/discover/categorie...,False,False,failed,2015-09-11 15:55:41,0.655304,"{""web"":{""project"":""https://www.kickstarter.com...",9.829555,domestic


In [4]:
print(df.shape, "\n",df.columns)

(3429, 27) 
 Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [5]:
# 1. Get main urls for all projects
urls = df.urls

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = urls_

# 2. Get urls for updates page
updates_urls = []

for i in range(0, len(urls)):
    split_string = urls[i].split("?", 1)
    updates_url = split_string[0] + "/posts"
    updates_urls.append(updates_url)

print(len(updates_urls))

3429


In [22]:
ids = df.id
start_int, end_int = 150,300

In [23]:
# Run these two lines if chromedriver does not execute (and comment out path and driver lines)
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

path = "Webdriver/chromedriver.exe"
driver = webdriver.Chrome(executable_path = path)

n_updates_all = []
updates_all = []
errors_url = []

for i in tqdm(range(start_int, end_int)):
    driver.get(updates_urls[i])

    # 1. Get total number of updates
    n_update = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="updates-emoji"]/span')))
    
    for number in n_update:
        n_updates_all.append(number.text)
        
    # 2. Get all updates and save as a list
    updates = []
    try: 
        elements = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,"//article")))
        for element in elements:
            updates.append(element.text)

    except TimeoutException:
        errors_url.append(updates_urls[i])
        pass
    
    updates_all.append(updates)

driver.quit()

 24%|███████████████████▍                                                             | 36/150 [05:49<18:25,  9.70s/it]


KeyboardInterrupt: 

In [ ]:
print('completed for index:', start_int, ' to ', end_int)
print('zeros:', len(errors_url))
print('expected: ', end_int-start_int)
print('success: ', len(n_updates_all), len(updates_all))

In [ ]:
df_export = pd.DataFrame({"id":ids.iloc[start_int:end_int],
                         "update":updates_all,
                         "n_updates":n_updates_all}, index = None)
filename = "Updates_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/Update/{}.csv".format(filename))

In [ ]:
%%notify
print('Scraping completed.')

## Draft Ignore

In [11]:
# from selenium.common.exceptions import TimeoutException
# from selenium.webdriver.common.action_chains import ActionChains

# # Run these two lines if chromedriver does not execute (and comment out path and driver lines)
# # from webdriver_manager.chrome import ChromeDriverManager
# # driver = webdriver.Chrome(ChromeDriverManager().install())

# path = "Webdriver/chromedriver.exe"
# driver = webdriver.Chrome(executable_path = path)


# updates_all = []
# errors_url = []

# for i in tqdm(range(5,10)):
#     driver.get(updates_urls[i])

#     # 1. Load all results
# #     while True:
# #         try:
# #             ActionChains(driver).move_to_element(WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load more')]")))).pause(5).click().perform()
# #     #         print("LOAD MORE button clicked")
# #         except TimeoutException:
# #     #         print("No more LOAD MORE button to be clicked")
# #             break

#     # 2. Get all updates and save as a list
#     updates = []
#     try: 
#         elements = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,"//article")))
#         for element in elements:
#             updates.append(element.text)

#     except TimeoutException:
#         errors_url.append(urls[i])
    
#     updates_all.append(updates)

# driver.quit()